# Trade
Loads valuation model(s) and applies error handling to rank sectors and trade

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import keras
import seaborn as sns
import matplotlib.pyplot as plt
from keras import layers
import pandas_datareader as pdr
from datetime import datetime
from keras.models import load_model
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform
import itertools
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform


Using TensorFlow backend.


In [2]:

NUM_INPUT_NEURONS = 64
NUM_OUTPUT_NEURONS = 1
DAY_OFFSET = 5

def from_network(symbol):
    return pdr.get_data_yahoo(symbols=symbol, start=datetime(1900, 1, 1)).sort_values(by=['Date'],ascending=False)

# Create features (only close price for now)
def convert_to_percentage(old, new):
    return (old - new) / old

def convert_labels_to_category(labels): 
    # Simplification - If positive return, 1, else 0
    # return map(lambda arr: 1 if arr[0] > 1 else 0, labels)
    # rounding simpliciation
    return map(lambda arr: map(lambda val: round(val,4),arr), labels)

def convert_to_train(raw_dataset, offset=5):
    dataset = raw_dataset.copy()
    features = []
    labels = []
    for i in range(offset, len(dataset) - NUM_INPUT_NEURONS):

        feature_dataset = dataset[i:i+NUM_INPUT_NEURONS].copy()
        latest_close = feature_dataset['Close'].iloc[0]
        
        features.append(
            feature_dataset['Close']
                .map(lambda current: convert_to_percentage(latest_close, current))
                .tolist()
        )
        labels.append([
            dataset['Close'].iloc[i-1] / latest_close, # 1 day trade
        ])
        
    # Without converting labels the precision is hard to determine accuracy. 
    # Rather than crude 0/1, maybe this can be more sophisticated
    labels = convert_labels_to_category(labels)
    
    return [features,labels]

def split_data(symbol):
    fetched = from_network(symbol)
    converted = convert_to_train(fetched)
    features = converted[0]
    labels = converted[1]
    training = [ # since we decide a model, use all data for training
        features[1:],
        labels[1:]
    ]
    validation = [
        features[5:1000],
        labels[5:1000]
    ]
    prediction = [
        features[:5],
        labels[:5],
    ]
    
    return {
        'symbol': symbol,
        'prediction': prediction,
        'validation': validation,
        'training': training,
    }

    
def combine_all(accum, prep):
    return {   
        'prediction':[
            accum['prediction'][0] + prep['prediction'][0],
            accum['prediction'][1] + prep['prediction'][1],
        ],
        'validation':[
            accum['validation'][0] + prep['validation'][0],
            accum['validation'][1] + prep['validation'][1],
        ],
        'training':[
            accum['training'][0] + prep['training'][0],
            accum['training'][1] + prep['training'][1],
        ],
    }


In [3]:
with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
    error_model = load_model('sector_model_error.h5')
    model = load_model('sector_model.h5')
    model_0 = load_model('sector_model_0.h5')
    model_1 = load_model('sector_model_1.h5')
    model_2 = load_model('sector_model_2.h5')
    model_3 = load_model('sector_model_3.h5')
    model_4 = load_model('sector_model_4.h5')
    model_5 = load_model('sector_model_5.h5')
    model_6 = load_model('sector_model_6.h5')
    model_7 = load_model('sector_model_7.h5')
    model_8 = load_model('sector_model_8.h5')
    model_9 = load_model('sector_model_9.h5')


In [4]:

QQQ = from_network('QQQ')
SPY = from_network('SPY')
XLK = from_network('XLK')
XLF = from_network('XLF')
XLE = from_network('XLE')
XLP = from_network('XLP')
XLV = from_network('XLV')
XLY = from_network('XLY')
XLI = from_network('XLI')
XLU = from_network('XLU')
DIA = from_network('DIA')
IWM = from_network('IWM')

In [19]:
securities_to_predict = [
#     ['SPY', np.array(convert_to_train(SPY.copy(), 0)[0][:1])],
#     ['QQQ', np.array(convert_to_train(QQQ.copy(), 0)[0][:1])],
    ['XLE', np.array(convert_to_train(XLE.copy(), 0)[0][:1])],
    ['XLF', np.array(convert_to_train(XLF.copy(), 0)[0][:1])],
    ['XLK', np.array(convert_to_train(XLK.copy(), 0)[0][:1])],
    ['XLP', np.array(convert_to_train(XLP.copy(), 0)[0][:1])],
    ['XLV', np.array(convert_to_train(XLV.copy(), 0)[0][:1])],
    ['XLU', np.array(convert_to_train(XLU.copy(), 0)[0][:1])],
    ['XLY', np.array(convert_to_train(XLY.copy(), 0)[0][:1])],
    ['XLI', np.array(convert_to_train(XLI.copy(), 0)[0][:1])],
#     ['IWM', np.array(convert_to_train(IWM.copy(), 0)[0][:1])],
#     ['DIA', np.array(convert_to_train(DIA.copy(), 0)[0][:1])],

]


In [20]:
def print_arg(arr):
    print '{} {}'.format(arr[0],str(arr[1]))
    
    
def predict_and_correct(model, inputs):
    prediction = model.predict(inputs)
    return prediction

def generate_model_predictions(model):
    predictions = map(lambda arr: [arr[0], predict_and_correct(model,arr[1])], securities_to_predict)
    predictions.sort(key=lambda x: x[1],reverse=True)
#     print map(print_arg, predictions)
    print map(lambda arr: arr[0],predictions)
    return predictions
    

In [21]:
all_model_predictions = map(generate_model_predictions,[model, model_1, model_2, model_3, model_4,model_5,model_6,model_7,model_8,model_9,model_0])


['XLY', 'XLK', 'XLE', 'XLP', 'XLI', 'XLF', 'XLU', 'XLV']
['XLE', 'XLF', 'XLI', 'XLV', 'XLY', 'XLU', 'XLP', 'XLK']
['XLP', 'XLI', 'XLK', 'XLU', 'XLY', 'XLE', 'XLV', 'XLF']
['XLK', 'XLF', 'XLP', 'XLV', 'XLE', 'XLY', 'XLI', 'XLU']
['XLE', 'XLU', 'XLV', 'XLK', 'XLY', 'XLP', 'XLI', 'XLF']
['XLI', 'XLY', 'XLK', 'XLP', 'XLU', 'XLE', 'XLV', 'XLF']
['XLE', 'XLV', 'XLY', 'XLK', 'XLP', 'XLF', 'XLI', 'XLU']
['XLU', 'XLF', 'XLP', 'XLV', 'XLY', 'XLI', 'XLE', 'XLK']
['XLE', 'XLI', 'XLK', 'XLF', 'XLV', 'XLY', 'XLU', 'XLP']
['XLV', 'XLY', 'XLK', 'XLF', 'XLP', 'XLU', 'XLE', 'XLI']
['XLP', 'XLU', 'XLV', 'XLF', 'XLI', 'XLK', 'XLY', 'XLE']


In [23]:

def ranking_for_symbol(symbol):
    return map(lambda predictions: map(lambda arr: arr[0],predictions).index(symbol),all_model_predictions)


securities_to_predict.sort(key=lambda x: np.median(ranking_for_symbol(x[0])))

for security in securities_to_predict:
    print security[0],np.median(ranking_for_symbol(security[0])),ranking_for_symbol(security[0])
        

XLK 2.0 [1, 7, 2, 0, 3, 2, 3, 7, 2, 2, 5]
XLF 3.0 [5, 1, 7, 1, 7, 7, 5, 1, 3, 3, 3]
XLP 3.0 [3, 6, 0, 2, 5, 3, 4, 2, 7, 4, 0]
XLV 3.0 [7, 3, 6, 3, 2, 6, 1, 3, 4, 0, 2]
XLE 4.0 [2, 0, 5, 4, 0, 5, 0, 6, 0, 6, 7]
XLY 4.0 [0, 4, 4, 5, 4, 1, 2, 4, 5, 1, 6]
XLI 4.0 [4, 2, 1, 6, 6, 0, 6, 5, 1, 7, 4]
XLU 5.0 [6, 5, 3, 7, 1, 4, 7, 0, 6, 5, 1]



# Trade policy

1. Long rank <=2
2. Short rank >=5
